In [1]:
from drowsiness_detection.data import load_raw_60_sec_data
import numpy as np
from drowsiness_detection import config
from joblib import dump
import time
from drowsiness_detection.models import build_cnn_model

In [2]:
data = load_raw_60_sec_data()

data.X_train = np.concatenate([data.X_train, data.X_val])
data.y_train = np.concatenate([data.y_train, data.y_val])


In [3]:
model_name = "cnn"

best_params = {
    "input_shape": (20, 1800, 7),
    "dropout_rate": 0.725,
    "kernel_size": 5,
    "learning_rate": 0.009,
    "num_conv_layers": 2,
    "num_filters": 44,
    "pooling": "max",
    "stride": 3
  }
model = build_cnn_model(**best_params)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1800, 7)]         0         
                                                                 
 conv1d (Conv1D)             (None, 600, 44)           1584      
                                                                 
 batch_normalization (BatchN  (None, 600, 44)          176       
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 600, 44)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 200, 44)           9724      
                                                                 
 batch_normalization_1 (Batc  (None, 200, 44)          176       
 hNormalization)                                             

In [4]:
history = model.fit(x=data.X_train, y=data.y_train, validation_data=(data.X_test, data.y_test) , verbose=1, class_weight={0: 0.84, 1: 1.14},
                    epochs=150, batch_size=20)


Epoch 1/150
1283/1283 [==============================] - 25s 19ms/step - loss: 0.6718 - accuracy: 0.5733 - val_loss: 0.6795 - val_accuracy: 0.6328
Epoch 2/150
1283/1283 [==============================] - 23s 18ms/step - loss: 0.6585 - accuracy: 0.5447 - val_loss: 0.6986 - val_accuracy: 0.4185
Epoch 3/150
1283/1283 [==============================] - 20s 16ms/step - loss: 0.6531 - accuracy: 0.5267 - val_loss: 0.6993 - val_accuracy: 0.4195
Epoch 4/150
1283/1283 [==============================] - 19s 15ms/step - loss: 0.6406 - accuracy: 0.5755 - val_loss: 0.6643 - val_accuracy: 0.6239
Epoch 5/150
1283/1283 [==============================] - 19s 15ms/step - loss: 0.6293 - accuracy: 0.6026 - val_loss: 0.6628 - val_accuracy: 0.6336
Epoch 6/150
1283/1283 [==============================] - 19s 15ms/step - loss: 0.6169 - accuracy: 0.6169 - val_loss: 0.9343 - val_accuracy: 0.6817
Epoch 7/150
1283/1283 [==============================] - 19s 15ms/step - loss: 0.6010 - accuracy: 0.6327 - val_loss: 0

In [5]:
time_string = time.strftime('%Y%m%d-%H%M%S')
model.save(str(config.MODEL_DIR_PATH.joinpath(f"{model_name}_{time_string}.h5")))


In [6]:
dump(history, str(config.MODEL_DIR_PATH.joinpath(f"histories/{model_name}_{time_string}.joblib")))

INFO:tensorflow:Assets written to: ram://323a35cf-2293-4290-8b14-8560887b14d7/assets


['/home/tim/IM/data/models/60sec/histories/cnn_20220728-212244.joblib']

In [7]:
y_pred_test = model.predict(data.X_test)
y_pred_train = model.predict(data.X_train)

In [8]:
print(np.mean((y_pred_test > .5) == data.y_test))


0.43691542947280426


In [9]:
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_test_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_test)
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_train_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_train)


In [10]:
exit()
